# PMSM with Inter-Turn Short Fault: Flux Map Identification

This notebook focuses on the identification of flux maps in Permanent Magnet Synchronous Motors (PMSM) with inter-turn short faults.
 The identified flux maps will be used for future fault diagnosis, enabling the detection and analysis of faults in PMSM systems. 
 This process is crucial for maintaining the reliability and efficiency of PMSM in various applications.

## Load important libraries

In [1]:
import  matplotlib.pyplot    as plt
import  numpy                as np   
import  multiprocessing      as mp
import  time
import  aut_lib

fem_path = 'C:/femm42'
pi = np.pi

## Simulation setup

In [2]:
# rotate the machine if neccessary
pp = 4
pi = np.pi
angle=0
ar=angle*pp*pi/180 # rotation in electrical angle for transformation

i_nom_rms=250 # [A] - nominal rms current

# setup id current range for the simulation
i_d_min=-1*i_nom_rms
i_d_max=1*i_nom_rms
cd=50 # [A] - increment
arraydim_d = int(((i_d_max + cd) - i_d_min)/cd)
iter_const_d = arraydim_d - 1

# setup iq current range for the simulation
i_q_min=-1*i_nom_rms
i_q_max=1*i_nom_rms
cq=50 # [A] - increment
arraydim_q = int(((i_q_max + cq) - i_q_min)/cq)
iter_const_q = arraydim_q - 1

# setup ifcurrent range for the simulation
i_f_min = -5*i_nom_rms
i_f_max =  5*i_nom_rms
cf = 250
arraydim_f = int(((i_f_max + cf) - i_f_min)/cf)
iter_const_f = arraydim_f - 1


i_dq=np.zeros((2,1))
i_abc=np.zeros((3,1))

# setup dimensions for angle dependency

angle_max = 90
angle_inc = 1.5
angle_dim = 1 # int((angle_max-0)/angle_inc + 1)
angle_const = angle_dim - 1 
angle_storage = np.zeros([angle_dim,])

# initialize flux ttables with zero
Fd  =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))
Fq  =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))
Fd2  =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))
Fq2  =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))
Ff  =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))
Iaf =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))

Br  =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))
Bt  =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))
M   =   np.zeros((arraydim_d,arraydim_q,arraydim_f,angle_dim))

j = 0
k = 0
l = 0
m = 0
# calculate and display the number of iterations of the simulation cycle
sim_start_time = time.time()
angle_act_el = 90
ar=angle_act_el*pi/180 # rotation in electrical angle for transformation

## Main part: execute simulations on parallel cores

In [ ]:
if __name__ == '__main__':
    # perform parallel computations  
    iterations = angle_dim*arraydim_d*arraydim_q
    iter_time_const = 35
    print("Number of total iterations:" + str(iterations))
    time_estimate =  (iterations*iter_time_const) / 3600
    print("The estimated time of simulation: " + str(time_estimate) + ' hours and ' + str(time_estimate/24) + ' days')
    for ang in range(0,angle_dim,1):
        for id in range(i_d_min,i_d_max + cd,cd):
            for iq in range(i_q_min,i_q_max + cq,cq):
                print('Id,Iq and angle is:' )
                print(id,iq,ang)
                iter_time_start = time.time()
                # create pools based on inner loop(fault-flux) dimension
           
                time_s = time.time()
                tasks = [(id, iq, i0, (ang * angle_inc), pp) for i0 in range(i_f_min, i_f_max + cf, cf)]
                num_cores = mp.cpu_count()
                resource_usage = 0.333
                nr_of_pools = min(len(tasks), arraydim_f)
                
                with mp.Pool(processes=round(resource_usage*num_cores)) as pool:
                    results = pool.starmap(aut_lib.calc_FEMM_faulted, tasks)
                    pool.close()
                    pool.join()


                print('Total time: %f seconds'%(time.time()-time_s))  

                # process results
                results = np.array(results)
                #print(iter_time_end - iter_time_start)
                Fd[k,l,:,j]     = results[:,0]
                Fq[k,l,:,j]     = results[:,1]
                Ff[k,l,:,j]     = results[:,2]
                Iaf[k,l,:,j]    = results[:,3]
                Br[k,l,:,j]     = results[:,4]
                Bt[k,l,:,j]     = results[:,5]
                M[k,l,:,j]      = results[:,6]
                Fd2[k,l,:,j]     = results[:,0]
                Fq2[k,l,:,j]     = results[:,1]
                print('ITER__END')
                l = l + 1
            k = k + 1
            l = 0
        j = j + 1
        k = 0


    id_grid     =   np.linspace(i_d_min,i_d_max,arraydim_d)
    iq_grid     =   np.linspace(i_q_min,i_q_max,arraydim_q)  
    if_grid     =   np.linspace(i_f_min,i_f_max,arraydim_f)      
    theta_grid  =   np.linspace(0,90,angle_dim)  * pp * pi /180 

    savemat('flux_maps_faulted.mat', {'psid': Fd , 'psiq': Fq,'psif': Ff,'id_grid':id_grid,'iq_grid':iq_grid,'if_grid':if_grid,'theta_grid':theta_grid, 'i_af_points' : Iaf,'b_rad' : Br,'b_tan' : Bt,'m_tot' : M,'psid2': Fd2,'psiq2': Fq2})
   


Number of total iterations:121
The estimated time of simulation: 1.176388888888889 hours and 0.04901620370370371 days
Id,Iq and angle is:
-250 -250 0
Total time: 37.137513 seconds
ITER__END
Id,Iq and angle is:
-250 -200 0
Total time: 31.962199 seconds
ITER__END
Id,Iq and angle is:
-250 -150 0
Total time: 33.454659 seconds
ITER__END
Id,Iq and angle is:
-250 -100 0
Total time: 43.039455 seconds
ITER__END
Id,Iq and angle is:
-250 -50 0
Total time: 53.168406 seconds
ITER__END
Id,Iq and angle is:
-250 0 0


## Visualization of final maps

In [ ]:

# Visualize flux tables
xd,yd = np.meshgrid(np.linspace(i_d_min,i_d_max,arraydim_d),np.linspace(i_q_min,i_q_max,arraydim_q))
zd = np.transpose(Fd[:,:,0,0])

xq,yq = np.meshgrid(np.linspace(i_d_min,i_d_max,arraydim_d),np.linspace(i_q_min,i_q_max,arraydim_q))
zq = np.transpose(Fq[:,:,0,0])

xf,yf = np.meshgrid(np.linspace(i_d_min,i_d_max,arraydim_d),np.linspace(i_q_min,i_q_max,arraydim_q))
zf = np.transpose(Ff[:,:,0,0])


# Create two subplots in a 1x2 grid
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, subplot_kw={'projection': '3d'})

# Plot the first surface on the left subplot
ax1.plot_surface(xd, yd, zd, cmap='viridis')
ax1.set_title(r'Flux map of $\Psi_d$ [Wb]')
ax1.set_xlabel('$i_d$ [A]')  # Set x label for the first subplot
ax1.set_ylabel('$i_q$ [A]')  # Set y label for the first subplot

# Plot the second surface on the right subplot
ax2.plot_surface(xq, yq, zq, cmap='viridis')
ax2.set_title(r'Flux map of $\Psi_q$ [Wb]')
ax2.set_xlabel('$i_d$ [A]')  # Set x label for the second subplot
ax2.set_ylabel('$i_q$ [A]')  # Set y label for the second subplot

# Plot the second surface on the right subplot
ax3.plot_surface(xf, yf, zf, cmap='viridis')
ax3.set_title(r'Flux map of $\Psi_f$ [Wb]')
ax3.set_xlabel('$i_d$ [A]')  # Set x label for the second subplot
ax3.set_ylabel('$i_q$ [A]')  # Set y label for the second subplot

plt.show()